In [4]:
from google.colab import files
uploaded = files.upload()

import pandas as pd
import numpy as np
import time
dataset = pd.read_csv("dataset60.csv")
dataset2 = np.array(dataset)

def normalization(dataset2) :
  (row,col) = dataset2.shape
  mins = np.zeros(shape=(col))
  maxs = np.zeros(shape=(col))
  for j in range (col) :
    mins[j] = np.min(dataset2[:,j])
    maxs[j] = np.max(dataset2[:,j])
    
  normalisasi = np.copy(dataset2)
  for i in range (row) :
    for j in range (col) :
      normalisasi[i,j] = ((dataset2[i,j]-mins[j]) / (maxs[j]-mins[j]))
  return normalisasi

normalisasi = normalization(dataset2)

training_data = np.vstack(normalisasi[0:48,0:3])
testing_data = np.vstack(normalisasi[48:60,0:3])

Saving dataset60.csv to dataset60.csv


In [5]:
str = input("number of centers/clusters: ")
l = int(str)

def ChooseICenters(training_data, l):
    centers = training_data[np.random.choice(training_data.shape[0], l, replace = False)]
    return(centers)

def EuclideanDistance(unit, center):
    baris = len(unit)
    sum   = 0
    for i in range(baris):
        sum += (unit[i] - center[i]) ** 2
    return np.sqrt(sum)

def AssigntoClusters(training_data, centers, clustering): 
    clusters           = np.copy(clustering) 
    (baris, kolom)     = training_data.shape
    p                  = len(centers)
    
    EuclideanDistances = np.zeros(shape = (p), dtype = np.float32)
    for i in range(baris): 
        for k in range(p): 
            EuclideanDistances[k] = EuclideanDistance(training_data[i], centers[k]) 
        criterion    = np.argmin(EuclideanDistances) 
        clusters[i]  = criterion
            
    if np.array_equal(clustering, clusters): 
        return False
    
    calc = np.zeros(shape = (p), dtype = np.int) 
    for i in range(baris): 
        number = clustering[i]
        calc[number] += 1 
    
    for k in range(p):  
        if calc[k] == 0:  
            return False
        
    for i in range(baris):
        clustering[i] = clusters[i] 
    return True 

def RecomputeCenters(training_data, centers, clustering):
    (baris, kolom) = training_data.shape
    p              = len(centers)
    calc           = np.zeros(shape = (p), dtype = np.int) 
    newcenters     = np.zeros(shape = centers.shape, dtype = np.float32) 
    for i in range(baris):
        number        = clustering[i]
        calc[number] += 1
        for j in range(kolom):
            newcenters[number,j] += training_data[i,j]
            
    for k in range(p):
        for j in range(kolom):
            newcenters[k,j] /= calc[k]
            
    for k in range(p):
        for j in range(kolom):
            centers[k,j] = newcenters[k,j]

def TemporaryCenters(training_data, p):
    (baris, kolom) = training_data.shape
    clustering     = np.zeros(shape = (baris), dtype = np.int) 
    centers        = np.zeros(shape = (p,kolom), dtype = np.float32)
    for k in range(p): 
        clustering[k] = k
    for k in range(p, baris):
        clustering[k] = np.random.randint(0,p)
    RecomputeCenters(training_data, centers, clustering)  
    return(clustering, centers)

def UpdatingCenters(training_data, p):
    (clustering, centers) = TemporaryCenters(training_data, p)
    
    term         = True 
    maxiteration = 100 
    epoch        = 1
    while epoch < maxiteration:
        epoch += 1 
        term = AssigntoClusters(training_data, centers, clustering)
        if term == False:
            break
        RecomputeCenters(training_data, centers, clustering)
    return(clustering, centers, epoch)

import math
check=True
while check==True:
    updatecenters = UpdatingCenters(training_data, l)
    centers       = updatecenters[1]
    check         = math.isnan(sum(sum(centers)))

#icenters            = ChooseICenters(training_data, l)
#temporarycenters    = TemporaryCenters(training_data, l)
#tcenters            = temporarycenters[1]
#tclusters            = temporarycenters[0]
print('========================================')
print('none nan')
print(centers)

number of centers/clusters: 3
none nan
[[0.22886406 0.50238097 0.05623471]
 [0.65283316 0.45408162 0.29811385]
 [0.27712438 0.8533834  0.6230858 ]]
